<a href="https://colab.research.google.com/github/zasuo-michikusa/Google_Colab_StableDiffusion/blob/main/StableDiffusion_AnimateDiff_custom_ipynb_20240103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

★事前準備★

ご自分の「Googleドライブ」の「MyDrive」に 以下のフォルダ構成と

任意でGoogleColab上の「Stable-Diffusion_WEB-UI（Forge）」で使用したいデータを格納してください。

MyDrive

└StableDiffusion

 　└checkpoints　※使用したいモデルデータを格納

 　└controlnet　※ツール用モデルを格納

 　└embeddings　※EasyNegativeなどを格納

 　└lora　※使用したいLoRAデータを格納

 　└VAE　※使用したいVAEデータを格納

 　└outputs　※生成データのコピー用受け皿フォルダ（フォルダのみでOK）

★このノートブックの使い方★

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～

①まずメニュー項目の「ランタイム」から
 → ランタイムの変更（いずれかのGPUタイプを選択）

②【01】の実行ボタン（▶）を押す、

（▶）の左に✅が付いたら次の（▶）を順番に押していく

※または「ランタイム」から「すべてのセルを実行」を実行する

③【06】の処理が進み、 https://************.gradio.live
と表示されたら、そのURLをクリック

④WEB-UIを停止するときは【05】の（▶）を押して処理をクルクルを停止

⑤再度WEB-UIを使用する際は、【05】の（▶）を押し③と同様

⑥生成したデータを保存する場合は【06】の（▶）を実行

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～


以下、「StableDiffusion_AnimateDiff環境」の起動手順です。

👇【01】Googleドライブのデータをこのノートブックへマウント（接続）

※Googleドライブにある、checkpoint、LoRA、VAE、Embedding、ControlNet用のツールモデル等を参照できるようにする

↓の（▶）を押し、出現するポップアップウィンドウから

　 Googleドライブに接続 → 「アカウントの選択」 → 「アクセスを許可」

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


👇【02】WEB-UIと各種_拡張機能のクローン（起動前にあらかじめ導入）※お好みで増減可能

In [ ]:
# Stable Diffusion Web-UI Forge のクローン（インストール）
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git
%cd /content/stable-diffusion-webui-forge

# Styles-Editorのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/chrisgoringe/Styles-Editor.git
%cd /content/stable-diffusion-webui-forge

# adetailerのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Bing-su/adetailer.git
%cd /content/stable-diffusion-webui-forge

# aspect-ratio-helperのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git
%cd /content/stable-diffusion-webui-forge

# prompt-all-in-oneのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Physton/sd-webui-prompt-all-in-one.git
%cd /content/stable-diffusion-webui-forge

# ultimate-upscale-for-automatic1111のクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
%cd /content/stable-diffusion-webui-forge

# SegmentAnythingのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/continue-revolution/sd-webui-segment-anything.git
%cd /content/stable-diffusion-webui-forge

# AnimateDiff（Forge版）のクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/continue-revolution/sd-forge-animatediff.git /content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff
%cd /content/stable-diffusion-webui-forge

# Deforumのクローン
%cd /content/stable-diffusion-webui-forge/extensions
!git clone https://github.com/deforum-art/sd-webui-deforum.git
%cd /content/stable-diffusion-webui-forge


👇【03】MP4形式の出力用に「ffpeg」をインストール

In [ ]:
%cd /content/stable-diffusion-webui-forge/
!venv\Scripts\activate
!pip install imageio[ffmpeg]

👇【04】GoogleドライブへHugingFaceからDL

「AnimateDiff」用の「MotionModel」

「SegmentAnything」用の「sam」「GroundingDINO」

In [ ]:
%cd /content/stable-diffusion-webui-forge/

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_hs.safetensors?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sdxl_v10_beta.safetensors?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-animatediff/model

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/sam

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinB.cfg.py?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/GroundingDINO_SwinT_OGC.cfg.py?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/groundingdino_swint_ogc.pth?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://huggingface.co/ShilongLiu/GroundingDINO/resolve/main/groundingdino_swinb_cogcoor.pth?download=true --directory-prefix=/content/stable-diffusion-webui-forge/extensions/sd-webui-segment-anything/models/grounding-dino

!wget https://civitai.com/api/download/models/329995 --directory-prefix=/content/stable-diffusion-webui-forge/models/svd

!wget https://huggingface.co/stabilityai/stable-zero123/resolve/main/stable_zero123.ckpt?download=true --directory-prefix=/content/stable-diffusion-webui-forge/models/z123



👇【05】WEB-UIの起動（各種大容量データはGoogleドライブを参照する設定） ※起動オプション追記もお好みで可能

★ ↓の（▶）で WEB-UIの「起動」「停止」「再起動」を操作します。

In [ ]:
!python launch.py --share --no-half-vae --disable-nan-check\
  --ckpt-dir "/content/drive/MyDrive/StableDiffusion/checkpoints" \
  --embeddings-dir "/content/drive/MyDrive/StableDiffusion/embeddings" \
  --lora-dir "/content/drive/MyDrive/StableDiffusion/lora" \
  --vae-dir "/content/drive/MyDrive/StableDiffusion/vae" \
  --controlnet-dir "/content/drive/MyDrive/StableDiffusion/controlnet"

👇【06】いったん【05】の処理を停止させた後に（▶）を実行してください。

生成したデータ「Outputデータ」をGoogleドライブへコピーします。

≪注意≫フォルダごと追加するので、フォルダ構成が多重化していきます。

※こまめにGoogleドライブ内の「outputs」フォルダのデータをローカルに移動と削除をしてください。

In [ ]:
!cp -r "/content/stable-diffusion-webui-forge/outputs/txt2img-grids" "/content/drive/MyDrive/StableDiffusion/outputs/txt2img-grids"
!cp -r "/content/stable-diffusion-webui-forge/outputs/txt2img-images" "/content/drive/MyDrive/StableDiffusion/outputs/txt2img-images"

cp: cannot stat '/content/stable-diffusion-webui-forge/outputs/txt2img-grids': No such file or directory
cp: cannot stat '/content/stable-diffusion-webui-forge/outputs/txt2img-images': No such file or directory
